VERSION 1

By: Murilo Farias

The code below goes to the VAM folder, analyzes all the subfolders, identifies which files are PDFs, and extracts them to the ALL_PDF folder.
If there are two files with the same name, it keeps the newest one.

os: Provides functions to interact with the operating system (e.g., file paths, checking file properties).
shutil: Used for file operations like copying files.
main_folder specifies the directory where the search for PDF files will begin. The path is a string and uses the r prefix to make it a raw string (to avoid issues with backslashes).
ALL_PDF inside the main_folder if it does not already exist.
os.path.join() is used to combine main_folder with ALL_PDF to create the full path for the destination folder.
os.makedirs(destination_folder, exist_ok=True) ensures that the folder is created, and if it already exists, it won't raise an error.
os.walk(main_folder) generates the file names in the directory tree rooted at main_folder, walking through every subdirectory (root), subdirectories (dirs), and files (files).
The if condition filters out only the files that have the .pdf extension (case-insensitive).
source_file: Full path of the PDF file found in the current subfolder.
destination_file: Full path where the PDF file would go in the destination folder
os.path.exists(destination_file) checks if the file already exists in the destination folder.
If it does exist, the script compares the last modified time of both files using os.path.getmtime().
If the source file is newer (source_modified_time > destination_modified_time), the file is copied to replace the older one in the destination folder.
The destination folder is always named ALL_PDF

1 Define the path to the main folder where all your files are stored (e.g., main_folder)
2 Create a folder (ALL_PDF) inside the main folder to store all the consolidated PDF files
3 Use os.walk() to iterate through all subfolders and files within the main_folder.
4 Check if each file ends with .pdf (case-insensitive).
5 If the PDF exists in the ALL_PDF folder, compare the last modified times of the source and destination files.
6 Copy or replace (if is newer, replace the older ;  if is older, skip the copy process ; If the file doesn't exist in the destination folder, copy it there.)
7 Print messages to indicate whether a file was copied, replaced, or skipped.
8 Once all PDFs are processed, print a final message confirming that all PDFs have been copied to the destination folder.

In [ ]:
import os
import shutil

# Set the path to your main folder
main_folder = r"C:\Users\MuriloFarias\OneDrive - Kubik Engenharia Ltda\Área de Trabalho\VAM"

# Create a destination folder to store all PDFs with a fixed name
destination_folder = os.path.join(main_folder, "ALL_PDF")
os.makedirs(destination_folder, exist_ok=True)

# Walk through all subfolders in the main folder
for root, dirs, files in os.walk(main_folder):
    # Skip the ALL_PDF folder
    if destination_folder in root:
        continue

    for file in files:
        if file.lower().endswith(".pdf"):  # Check if the file is a PDF
            source_file = os.path.join(root, file)
            destination_file = os.path.join(destination_folder, file)

            if os.path.exists(destination_file):
                # Compare last modified time of both files
                source_modified_time = os.path.getmtime(source_file)
                destination_modified_time = os.path.getmtime(destination_file)

                if source_modified_time > destination_modified_time:
                    # Replace the older file in the destination folder
                    shutil.copy(source_file, destination_file)
                    print(f"Replaced older file: {file}")
                else:
                    print(f"Skipped older file: {file}")
            else:
                # Copy the file if it doesn't exist in the destination
                shutil.copy(source_file, destination_folder)
                print(f"Copied new file: {file}")

print(f"All PDFs have been copied to: {destination_folder}")


To analyze all PDFs in the ALL_PDF folder and extract the text while ignoring images, diagrams, and other non-text content, I'm using PyPDF2 
1 Install PyPDF2 for text extraction.
2 Use PyPDF2.PdfReader: Reads the PDF file and extracts text from each page.
3 The script loops through all PDFs in the ALL_PDF folder.
4 Extracts text from each PDF.
5 Saves the extracted text into a summary file (extracted_text_summary.txt) for easy review.
6 Catches and logs errors if any PDF cannot be processed.
7 Counts the total number of PDFs in the folder.
8 Displays the progress percentage and the current file being processed in real time.

In [4]:
pip install PyPDF2


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\MuriloFarias\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



   ---------------------------------------- 0.0/232.6 kB ? eta -:--:--
   - -------------------------------------- 10.2/232.6 kB ? eta -:--:--
   ------ -------------------------------- 41.0/232.6 kB 393.8 kB/s eta 0:00:01
   ------------ -------------------------- 71.7/232.6 kB 491.5 kB/s eta 0:00:01
   --------------------- ---------------- 133.1/232.6 kB 782.7 kB/s eta 0:00:01
   ---------------------------------------- 232.6/232.6 kB 1.1 MB/s eta 0:00:00


In [ ]:
import os
from PyPDF2 import PdfReader

# Path to the ALL_PDF folder
all_pdf_folder = r"C:\Users\MuriloFarias\OneDrive - Kubik Engenharia Ltda\Área de Trabalho\VAM\ALL_PDF"

# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_path):
    try:
        reader = PdfReader(pdf_path)
        text = ""
        for page in reader.pages:
            text += page.extract_text()  # Extract text from each page
        return text
    except Exception as e:
        print(f"Error processing {pdf_path}: {e}")
        return ""

# Analyze all PDFs in the folder
all_text_data = {}  # Dictionary to store PDF file names and their extracted text
for file_name in os.listdir(all_pdf_folder):
    if file_name.lower().endswith(".pdf"):  # Check if the file is a PDF
        pdf_path = os.path.join(all_pdf_folder, file_name)
        print(f"Extracting text from: {file_name}")
        extracted_text = extract_text_from_pdf(pdf_path)
        all_text_data[file_name] = extracted_text

# Optional: Save all extracted text to a summary file
summary_file_path = os.path.join(all_pdf_folder, "extracted_text_summary.txt")
with open(summary_file_path, "w", encoding="utf-8") as summary_file:
    for file_name, text in all_text_data.items():
        summary_file.write(f"--- {file_name} ---\n{text}\n\n")

print(f"Text extraction completed! Summary saved to: {summary_file_path}")


In [6]:
import os
import time
from PyPDF2 import PdfReader
from datetime import datetime

# Paths
main_folder = r"C:\Users\MuriloFarias\OneDrive - Kubik Engenharia Ltda\Área de Trabalho\VAM"
all_pdf_folder = os.path.join(main_folder, "ALL_PDF")
summary_file_path = os.path.join(main_folder, "extracted_text_summary.txt")  # Save summary in the main folder

# Start timer
start_time = time.time()

# Load already processed file names and modification dates from the summary file
processed_files = {}
if os.path.exists(summary_file_path):
    with open(summary_file_path, "r", encoding="utf-8") as summary_file:
        for line in summary_file:
            if line.startswith("---"):
                parts = line.strip("--- ").strip().split(" --- ")
                if len(parts) == 3:
                    file_name, file_path, mod_date = parts[0], parts[1], parts[2].replace("Data of file: ", "")
                    processed_files[file_name] = mod_date

# Get all PDF files in the folder
pdf_files = [f for f in os.listdir(all_pdf_folder) if f.lower().endswith(".pdf")]
total_files = len(pdf_files)

# Identify new or updated PDFs
new_files = []
for file_name in pdf_files:
    file_path = os.path.join(all_pdf_folder, file_name)
    mod_time = os.path.getmtime(file_path)  # Get last modification time
    mod_date = datetime.fromtimestamp(mod_time).strftime("%Y-%m-%d")

    if file_name not in processed_files or processed_files[file_name] < mod_date:
        new_files.append((file_name, file_path, mod_date))

# Process new or updated PDFs
processed_count = 0
if not new_files:
    print("No new or updated PDFs to process. All files are up to date!")
else:
    print(f"Found {len(new_files)} new or updated file(s) to process.")
    for index, (file_name, file_path, mod_date) in enumerate(new_files):
        print(f"Processing file {index + 1} of {len(new_files)}: {file_name}")
        
        # Extract text
        try:
            reader = PdfReader(file_path)
            text = ""
            for page in reader.pages:
                text += page.extract_text()

            # Append text to the summary file
            with open(summary_file_path, "a", encoding="utf-8") as summary_file:
                summary_file.write(f"--- {file_name} --- PATH {file_path} --- Data of file: {mod_date}\n{text}\n\n")

            processed_files[file_name] = mod_date
            processed_count += 1
            print(f"Processed: {file_name}")
        except Exception as e:
            print(f"Error processing {file_name}: {e}")

# End timer
end_time = time.time()
elapsed_time = end_time - start_time

# KPIs
success_rate = (processed_count / total_files * 100) if total_files > 0 else 0
failure_rate = 100 - success_rate

print("\n--- Processing Summary ---")
print(f"Run Time: {elapsed_time:.2f} minutes")
print(f"Total PDFs in ALL_PDF folder: {total_files}")
print(f"New or Updated PDFs processed: {processed_count}")
print(f"Success Rate: {success_rate:.2f}%")
print(f"Failure Rate: {failure_rate:.2f}%")


Found 479 new or updated file(s) to process.
Processing file 1 of 479: 123-176.pdf
Processed: 123-176.pdf
Processing file 2 of 479: 131-003.pdf
Processed: 131-003.pdf
Processing file 3 of 479: 183997_APROVAÇÃO_ST.ADELIA_DS-0414-B.pdf
Processed: 183997_APROVAÇÃO_ST.ADELIA_DS-0414-B.pdf
Processing file 4 of 479: 183997_APROVAÇÃO_ST.ADELIA_FD.pdf
Processed: 183997_APROVAÇÃO_ST.ADELIA_FD.pdf
Processing file 5 of 479: 23.2382.0005 - Cubículo de Conexão UFV - CPFL 15kV - Uso Externo - UFV SANTA ADÉLIA.pdf
Processed: 23.2382.0005 - Cubículo de Conexão UFV - CPFL 15kV - Uso Externo - UFV SANTA ADÉLIA.pdf
Processing file 6 of 479: 23.2382.0005.pdf
Processed: 23.2382.0005.pdf
Processing file 7 of 479: 23.2382.0006 - Cubículo de Conexão UFV - CPFL 15kV - Uso Externo - UFV SANTA ADÉLIA.pdf
Processed: 23.2382.0006 - Cubículo de Conexão UFV - CPFL 15kV - Uso Externo - UFV SANTA ADÉLIA.pdf
Processing file 8 of 479: 23.2382.0006.pdf
Processed: 23.2382.0006.pdf
Processing file 9 of 479: 23.2382.0007 - 

unknown widths : 
[0, IndirectObject(1673, 0, 1821560102608)]
unknown widths : 
[0, IndirectObject(1675, 0, 1821560102608)]
unknown widths : 
[0, IndirectObject(1677, 0, 1821560102608)]
unknown widths : 
[0, IndirectObject(1679, 0, 1821560102608)]
unknown widths : 
[0, IndirectObject(1681, 0, 1821560102608)]
unknown widths : 
[0, IndirectObject(1683, 0, 1821560102608)]
unknown widths : 
[0, IndirectObject(1685, 0, 1821560102608)]
unknown widths : 
[0, IndirectObject(1687, 0, 1821560102608)]
unknown widths : 
[0, IndirectObject(1689, 0, 1821560102608)]
unknown widths : 
[0, IndirectObject(1691, 0, 1821560102608)]
unknown widths : 
[0, IndirectObject(1693, 0, 1821560102608)]
unknown widths : 
[0, IndirectObject(1695, 0, 1821560102608)]
unknown widths : 
[0, IndirectObject(1697, 0, 1821560102608)]
unknown widths : 
[0, IndirectObject(1699, 0, 1821560102608)]
unknown widths : 
[0, IndirectObject(1701, 0, 1821560102608)]
unknown widths : 
[0, IndirectObject(1703, 0, 1821560102608)]
unknown 

Processed: ATO-COM-TRAFOS-DEODE-SA-UFV1-R00-Comentado Kubik.pdf
Processing file 42 of 479: ATO-COM-TRAFOS-DEODE-SA-UFV1-R00.pdf


unknown widths : 
[0, IndirectObject(1580, 0, 1821561510160)]
unknown widths : 
[0, IndirectObject(1586, 0, 1821561510160)]
unknown widths : 
[0, IndirectObject(1592, 0, 1821561510160)]
unknown widths : 
[0, IndirectObject(1598, 0, 1821561510160)]
unknown widths : 
[0, IndirectObject(1604, 0, 1821561510160)]
unknown widths : 
[0, IndirectObject(1610, 0, 1821561510160)]
unknown widths : 
[0, IndirectObject(1616, 0, 1821561510160)]
unknown widths : 
[0, IndirectObject(1622, 0, 1821561510160)]
unknown widths : 
[0, IndirectObject(1628, 0, 1821561510160)]
unknown widths : 
[0, IndirectObject(1634, 0, 1821561510160)]
unknown widths : 
[0, IndirectObject(1644, 0, 1821561510160)]
unknown widths : 
[0, IndirectObject(1650, 0, 1821561510160)]
unknown widths : 
[0, IndirectObject(1668, 0, 1821561510160)]
unknown widths : 
[0, IndirectObject(1674, 0, 1821561510160)]
unknown widths : 
[0, IndirectObject(1680, 0, 1821561510160)]
unknown widths : 
[0, IndirectObject(1686, 0, 1821561510160)]
unknown 

Processed: ATO-COM-TRAFOS-DEODE-SA-UFV1-R00.pdf
Processing file 43 of 479: ATO-COM-TRAFOS-DEODE-SA-UFV2-R00 - Comentado.pdf
Processed: ATO-COM-TRAFOS-DEODE-SA-UFV2-R00 - Comentado.pdf
Processing file 44 of 479: ATO-COM-TRAFOS-DEODE-SA-UFV2-R00.pdf
Processed: ATO-COM-TRAFOS-DEODE-SA-UFV2-R00.pdf
Processing file 45 of 479: ATO-COM-TRAFOS-DEODE-SA-UFV2-R02-Comentado Kubik.pdf


...

Processed: SPSTADE01-ENG-E3-DES-236-R01 - 23.2382.0010 - trfaux 2 kVA.pdf
Processing file 378 of 479: SPSTADE01-ENG-E3-DES-237-R01.pdf
Processed: SPSTADE01-ENG-E3-DES-237-R01.pdf
Processing file 379 of 479: SPSTADE01-ENG-E3-DES-237-R02 (Pond. Deode).pdf
Processed: SPSTADE01-ENG-E3-DES-237-R02 (Pond. Deode).pdf
Processing file 380 of 479: SPSTADE01-ENG-E3-DES-237-R02.pdf
Processed: SPSTADE01-ENG-E3-DES-237-R02.pdf
Processing file 381 of 479: SPSTADE01-ENG-E3-DES-237-R03- Comentado.pdf
Processed: SPSTADE01-ENG-E3-DES-237-R03- Comentado.pdf
Processing file 382 of 479: SPSTADE01-ENG-E3-DES-237-R03.pdf
Processed: SPSTADE01-ENG-E3-DES-237-R03.pdf
Processing file 383 of 479: SPSTADE01-ENG-E3-DES-237-R04.pdf
Processed: SPSTADE01-ENG-E3-DES-237-R04.pdf
Processing file 384 of 479: SPSTADE01-ENG-E3-DES-237-R05.pdf
Processed: SPSTADE01-ENG-E3-DES-237-R05.pdf
Processing file 385 of 479: SPSTADE01-ENG-E3-DES-239-R01 (Pond. Deode).pdf


Multiple definitions in dictionary at byte 0x9191a for key /PageMode


Processed: SPSTADE01-ENG-E3-DES-239-R01 (Pond. Deode).pdf
Processing file 386 of 479: SPSTADE01-ENG-E3-DES-239-R01.pdf
Processed: SPSTADE01-ENG-E3-DES-239-R01.pdf
Processing file 387 of 479: SPSTADE01-ENG-E3-DES-239-R02.pdf
Processed: SPSTADE01-ENG-E3-DES-239-R02.pdf
Processing file 388 of 479: SPSTADE01-ENG-E3-EST-120-R01.pdf
Processed: SPSTADE01-ENG-E3-EST-120-R01.pdf
Processing file 389 of 479: SPSTADE01-ENG-E3-EST-120-R01_APR.pdf
Processed: SPSTADE01-ENG-E3-EST-120-R01_APR.pdf
Processing file 390 of 479: SPSTADE01-ENG-E3-EST-235-R01 - D485-LCR.pdf
Processed: SPSTADE01-ENG-E3-EST-235-R01 - D485-LCR.pdf
Processing file 391 of 479: SPSTADE01-ENG-E3-EST-235-R01 - Datalogger-LCR.pdf
Processed: SPSTADE01-ENG-E3-EST-235-R01 - Datalogger-LCR.pdf
Processing file 392 of 479: SPSTADE01-ENG-E3-EST-235-R01 - Desenho Sensores Tracker-LCR.pdf
Processed: SPSTADE01-ENG-E3-EST-235-R01 - Desenho Sensores Tracker-LCR.pdf
Processing file 393 of 479: SPSTADE01-ENG-E3-EST-235-R01 - FA01-LCR.pdf
Processed

Multiple definitions in dictionary at byte 0x25a12e for key /PageMode


Processed: SPSTADE01-ENG-E3-MDC-231-R01.pdf
Processing file 457 of 479: SPSTADE01-ENG-E3-MDC-233-R01.pdf
Processed: SPSTADE01-ENG-E3-MDC-233-R01.pdf
Processing file 458 of 479: SPSTADE01-ENG-E3-MDC-233-R03 (Pond. Deode).pdf
Processed: SPSTADE01-ENG-E3-MDC-233-R03 (Pond. Deode).pdf
Processing file 459 of 479: SPSTADE01-ENG-E3-MDC-233-R03.pdf
Processed: SPSTADE01-ENG-E3-MDC-233-R03.pdf
Processing file 460 of 479: SPSTADE01-ENG-E3-MDC-233-R04.pdf
Processed: SPSTADE01-ENG-E3-MDC-233-R04.pdf
Processing file 461 of 479: SPSTADE01-ENG-E3-MDC-234-R01.pdf
Processed: SPSTADE01-ENG-E3-MDC-234-R01.pdf
Processing file 462 of 479: SPSTADE01-ENG-E3-MDC-234-R02 - Memorial Descritivo.pdf
Processed: SPSTADE01-ENG-E3-MDC-234-R02 - Memorial Descritivo.pdf
Processing file 463 of 479: SPSTADE01-ENG-E3-MDC-234-R02.pdf
Processed: SPSTADE01-ENG-E3-MDC-234-R02.pdf
Processing file 464 of 479: SPSTADE01-ENG-E3-MDC-238-R01.pdf
Processed: SPSTADE01-ENG-E3-MDC-238-R01.pdf
Processing file 465 of 479: SPSTADE01-ENG-E3

VERSION 2

In [2]:
pip install pdfplumber

     ---------------------------------------- 0.0/42.5 kB ? eta -:--:--
     ------------------ ------------------- 20.5/42.5 kB 640.0 kB/s eta 0:00:01
     -------------------------------------- 42.5/42.5 kB 511.8 kB/s eta 0:00:00
     ---------------------------------------- 0.0/48.2 kB ? eta -:--:--
     --------------------------------- ------ 41.0/48.2 kB 2.0 MB/s eta 0:00:01
     ---------------------------------------- 48.2/48.2 kB 1.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/59.5 kB ? eta -:--:--
   ---------------------------------------- 59.5/59.5 kB 3.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/5.6 MB ? eta -:--:--
   ------ --------------------------------- 0.9/5.6 MB 18.9 MB/s eta 0:00:01
   --------------------------- ------------ 3.8/5.6 MB 40.7 MB/s eta 0:00:01
   ---------------------------------------  5.6/5.6 MB 40.0 MB/s eta 0:00:01
   ---------------------------------------- 5.6/5.6 MB 35.9 MB/s eta 0:00:00
   --------


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\MuriloFarias\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
pip install sentence-transformers

     ---------------------------------------- 0.0/44.4 kB ? eta -:--:--
     --------- ------------------------------ 10.2/44.4 kB ? eta -:--:--
     ----------------------------------- -- 41.0/44.4 kB 487.6 kB/s eta 0:00:01
     -------------------------------------- 44.4/44.4 kB 437.8 kB/s eta 0:00:00
     ---------------------------------------- 0.0/57.7 kB ? eta -:--:--
     ---------------------------------------- 57.7/57.7 kB 1.5 MB/s eta 0:00:00
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ---------------------------------------- 60.8/60.8 kB 3.2 MB/s eta 0:00:00
     ---------------------------------------- 0.0/41.5 kB ? eta -:--:--
     ---------------------------------------- 41.5/41.5 kB ? eta 0:00:00
   ---------------------------------------- 0.0/268.8 kB ? eta -:--:--
   ------------------------------------ --- 245.8/268.8 kB 7.6 MB/s eta 0:00:01
   ---------------------------------------- 268.8/268.8 kB 4.1 MB/s eta 0:00:00
   ------------


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\MuriloFarias\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [6]:
pip install faiss-cpu

   ---------------------------------------- 0.0/13.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/13.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/13.8 MB 487.6 kB/s eta 0:00:29
   ---------------------------------------- 0.1/13.8 MB 744.7 kB/s eta 0:00:19
    --------------------------------------- 0.3/13.8 MB 2.0 MB/s eta 0:00:07
   --- ------------------------------------ 1.0/13.8 MB 4.7 MB/s eta 0:00:03
   --- ------------------------------------ 1.0/13.8 MB 4.7 MB/s eta 0:00:03
   ------ --------------------------------- 2.1/13.8 MB 7.0 MB/s eta 0:00:02
   ------- -------------------------------- 2.5/13.8 MB 7.1 MB/s eta 0:00:02
   --------- ------------------------------ 3.1/13.8 MB 8.4 MB/s eta 0:00:02
   ------------ --------------------------- 4.2/13.8 MB 9.5 MB/s eta 0:00:02
   ------------ --------------------------- 4.2/13.8 MB 9.5 MB/s eta 0:00:02
   --------------- ------------------------ 5.2/13.8 MB 9.8 MB/s eta 0:00:01
   ------


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\MuriloFarias\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [7]:
pip install faiss-gpu

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\MuriloFarias\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [9]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\MuriloFarias\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [10]:
import os
from pdfplumber import open as open_pdf
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

print("All libraries installed and working!")


C:\Users\MuriloFarias\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


All libraries installed and working!


Step 1: Extract Text and Generate Embeddings

In [1]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\MuriloFarias\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [10]:
import os
import torch
from pdfplumber import open as open_pdf
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import pickle
from tqdm import tqdm  # For progress bar

# Verify PyTorch Installation
try:
    print("PyTorch Version:", torch.__version__)
    print("CUDA Available:", torch.cuda.is_available())
except Exception as e:
    print("Error checking PyTorch:", e)

# Paths
pdf_folder = r"C:\Users\MuriloFarias\OneDrive - Kubik Engenharia Ltda\Área de Trabalho\VAM\SELECIONADOS"
save_folder = r"C:\Users\MuriloFarias\Documents\AI"  # Save FAISS index and metadata here
os.makedirs(save_folder, exist_ok=True)  # Ensure the save folder exists
index_path = os.path.join(save_folder, "faiss_index.bin")
metadata_path = os.path.join(save_folder, "metadata.pkl")

# Load the SentenceTransformer model
try:
    model = SentenceTransformer('all-MiniLM-L6-v2')
    print("SentenceTransformer model loaded successfully!")
except Exception as e:
    print("Error loading SentenceTransformer:", e)

# Initialize FAISS
dimension = 384  # Embedding dimension for all-MiniLM-L6-v2
index = faiss.IndexFlatL2(dimension)
metadata = []  # To track file and chunk details for embeddings

# Load existing metadata and index if they exist
if os.path.exists(metadata_path):
    with open(metadata_path, "rb") as f:
        metadata = pickle.load(f)
    print("Loaded existing metadata.")

if os.path.exists(index_path):
    index = faiss.read_index(index_path)
    print("Loaded existing FAISS index.")

# Counters for processed files
files_with_text = 0
files_without_text = 0
files_without_text_list = []

# Function to extract text from PDFs
def extract_text_from_pdf(file_path):
    with open_pdf(file_path) as pdf:
        text = ""
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text
    return text

# Function to chunk text
def chunk_text(text, max_length=300):
    """Split text into smaller chunks for better embedding."""
    sentences = text.split(".")
    chunks = []
    current_chunk = ""
    for sentence in sentences:
        if len(current_chunk) + len(sentence) < max_length:
            current_chunk += sentence + "."
        else:
            chunks.append(current_chunk)
            current_chunk = sentence + "."
    if current_chunk:
        chunks.append(current_chunk)
    return chunks

# Batch process embeddings
def process_embeddings_in_batches(chunks, batch_size=32):
    embeddings = []
    for i in range(0, len(chunks), batch_size):
        batch = chunks[i:i + batch_size]
        batch_embeddings = model.encode(batch, convert_to_tensor=True).cpu().numpy()
        embeddings.extend(batch_embeddings)
    return embeddings

# Get list of PDF files
pdf_files = [file for file in os.listdir(pdf_folder) if file.endswith(".pdf")]

# Process all PDFs with a progress bar
with tqdm(total=len(pdf_files), desc="Processing PDFs", unit="file") as pbar:
    for file_name in pdf_files:
        file_path = os.path.join(pdf_folder, file_name)
        try:
            # Extract text
            text = extract_text_from_pdf(file_path)
            if text.strip():
                files_with_text += 1
                chunks = chunk_text(text)  # Split text into chunks

                # Process embeddings in batches
                embeddings = process_embeddings_in_batches(chunks)
                index.add(np.array(embeddings))  # Add chunk embeddings to FAISS

                for chunk in chunks:
                    metadata.append({"file": file_name, "path": file_path, "chunk": chunk})
            else:
                files_without_text += 1
                files_without_text_list.append(file_name)
                print(f"No extractable text found in {file_name}. Images or empty pages may be present.")

            # Save periodically
            if len(metadata) % 1000 == 0:
                print("Saving intermediate FAISS index and metadata...")
                faiss.write_index(index, index_path)
                with open(metadata_path, "wb") as f:
                    pickle.dump(metadata, f)

            pbar.set_postfix({"Current File": file_name})
            pbar.update(1)  # Update progress bar

        except Exception as e:
            print(f"Error processing {file_name}: {e}")

# Final save of the FAISS index and metadata
try:
    print(f"Saving final FAISS index to: {index_path}")
    faiss.write_index(index, index_path)
    print("FAISS index saved successfully.")

    print(f"Saving final metadata to: {metadata_path}")
    with open(metadata_path, "wb") as f:
        pickle.dump(metadata, f)
    print("Metadata saved successfully.")
except Exception as e:
    print(f"Error saving FAISS index or metadata: {e}")

# Report results
print(f"Total files processed: {len(pdf_files)}")
print(f"Files with extractable text: {files_with_text}")
print(f"Files without extractable text: {files_without_text}")
if files_without_text_list:
    print("Files without extractable text:")
    for file_name in files_without_text_list:
        print(f"- {file_name}")


PyTorch Version: 2.5.1+cpu
CUDA Available: False
SentenceTransformer model loaded successfully!
Loaded existing metadata.
Loaded existing FAISS index.


Processing PDFs:   2%|▏         | 1/50 [00:00<00:29,  1.67file/s, Current File=123-176.pdf]

No extractable text found in 123-176.pdf. Images or empty pages may be present.


Processing PDFs: 100%|██████████| 50/50 [21:24<00:00, 25.69s/file, Current File=ATO-COM-TRAFOS-DEODE-SA-UFV3-R00.pdf]                                                                                          

Saving final FAISS index to: C:\Users\MuriloFarias\Documents\AI\faiss_index.bin
FAISS index saved successfully.
Saving final metadata to: C:\Users\MuriloFarias\Documents\AI\metadata.pkl
Metadata saved successfully.
Total files processed: 50
Files with extractable text: 49
Files without extractable text: 1
Files without extractable text:
- 123-176.pdf


saved in: C:\Users\MuriloFarias\Documents\AI

Step 2: Build a Query System with FAISS

In [11]:
import os
import faiss
import pickle
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.preprocessing import normalize

# Paths
save_folder = r"C:\Users\MuriloFarias\Documents\AI"
index_path = os.path.join(save_folder, "faiss_index.bin")
metadata_path = os.path.join(save_folder, "metadata.pkl")

# Load the SentenceTransformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Load FAISS index and metadata
index = faiss.read_index(index_path)
with open(metadata_path, "rb") as f:
    metadata = pickle.load(f)

# Normalize the FAISS index for cosine similarity
def normalize_index(index):
    embeddings = []
    for i in range(index.ntotal):
        vec = index.reconstruct(i)
        embeddings.append(vec)
    embeddings = np.array(embeddings)
    embeddings = normalize(embeddings, axis=1)  # Normalize to unit vectors
    new_index = faiss.IndexFlatIP(embeddings.shape[1])  # Cosine similarity
    new_index.add(embeddings)
    return new_index

index = normalize_index(index)

# Function to query FAISS
def query_faiss(question, index, model, metadata):
    # Generate embedding for the question
    question_embedding = model.encode([question], convert_to_tensor=True).cpu().numpy()
    question_embedding = normalize(question_embedding, axis=1)  # Normalize the question vector

    # Search FAISS for the most similar result
    D, I = index.search(np.array(question_embedding), k=1)  # k=1 for top result

    # If a result is found, return metadata
    if I[0][0] != -1:
        matched_metadata = metadata[I[0][0]]
        similarity_score = D[0][0]
        return {
            "relevant_file": matched_metadata["file"],
            "file_path": matched_metadata["path"],
            "similarity_score": similarity_score
        }
    else:
        return {"error": "No relevant information found."}



In [14]:
pip install flask

   ---------------------------------------- 0.0/103.0 kB ? eta -:--:--
   --- ------------------------------------ 10.2/103.0 kB ? eta -:--:--
   ----------------------- --------------- 61.4/103.0 kB 825.8 kB/s eta 0:00:01
   -------------------------------------- 103.0/103.0 kB 986.6 kB/s eta 0:00:00
   ---------------------------------------- 0.0/98.2 kB ? eta -:--:--
   ---------------------------------------- 98.2/98.2 kB 2.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/224.5 kB ? eta -:--:--
   --------------------------------------- 224.5/224.5 kB 13.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\MuriloFarias\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [15]:
pip install faiss-cpu


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\MuriloFarias\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


3 Create a Web Interface with Streamlit

In [30]:
pip install streamlit

Note: you may need to restart the kernel to use updated packages.Collecting streamlit
     ---------------------------------------- 0.0/44.3 kB ? eta -:--:--
     ---------------------------------------- 44.3/44.3 kB 2.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/9.1 MB ? eta -:--:--
   ---- ----------------------------------- 0.9/9.1 MB 20.0 MB/s eta 0:00:01
   ------------------ --------------------- 4.2/9.1 MB 45.1 MB/s eta 0:00:01
   ------------------------------- -------- 7.2/9.1 MB 51.6 MB/s eta 0:00:01
   ----------------------------------- ---- 8.1/9.1 MB 47.1 MB/s eta 0:00:01
   ---------------------------------------  9.1/9.1 MB 41.7 MB/s eta 0:00:01
   ---------------------------------------- 9.1/9.1 MB 38.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/731.2 kB ? eta -:--:--
   --------------------------------------- 731.2/731.2 kB 48.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/207.6 kB ? eta -:--:--
   ------


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\MuriloFarias\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [31]:
pip show streamlit


Name: streamlit
Version: 1.41.1
Summary: A faster way to build and share data apps
Home-page: https://streamlit.io
Author: Snowflake Inc
Author-email: hello@streamlit.io
License: Apache License 2.0
Location: C:\Users\MuriloFarias\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages
Requires: altair, blinker, cachetools, click, gitpython, numpy, packaging, pandas, pillow, protobuf, pyarrow, pydeck, requests, rich, tenacity, toml, tornado, typing-extensions, watchdog
Required-by: 
Note: you may need to restart the kernel to use updated packages.
